 # Sofaescore web scraper 

#### OBJETIVO: obtener el performance de todos los jugaodres de LaLiga en los ultimos 29 partidos

##### - dependencias:

In [1]:
import requests
import os
import regex 
import json
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException  # Importa la excepción necesaria
from bs4 import BeautifulSoup

In [2]:
headers = {
	"X-RapidAPI-Key": "11822210cdmsha855c4a12c471b5p18100fjsn3972b17b3be8",
	"X-RapidAPI-Host": "sofascore.p.rapidapi.com"
}

##### - función para guardar performance de un jugador en el archivo json del partido

In [3]:
def performance_to_json(JsonJugador):
    
    nombre_archivo = f"performance_jugadores_partido_{slugJson}.json"
    ruta_completa_archivo = os.path.join(ruta_jornada, nombre_archivo)
    
    # Verificar si el archivo JSON existe
    if os.path.exists(ruta_completa_archivo):
        # Si el archivo existe, cargar su contenido
        with open(ruta_completa_archivo, 'r') as archivo:
            datos = json.load(archivo)
    else:
        # Si el archivo no existe, crear un diccionario vacío
        datos = {}
    # Actualizar el diccionario existente con la nueva entrada
    datos.update(JsonJugador)
    
    # Guardar los datos actualizados en el archivo JSON
    if not os.path.exists(ruta_completa_archivo):
        with open(ruta_completa_archivo, 'w') as archivo:
            archivo.write("{}")  # Crea un archivo vacío si no existe
    with open(ruta_completa_archivo, 'w') as archivo:
        json.dump(datos, archivo, indent=4)

##### - función para extraer todos los datos del performance de un jugador

In [4]:
def obtener_informacion_jugador():

    # Obtiene el contenido HTML de la página
    pagina_html = driver.page_source
    #print(pagina_html)

    # Utiliza BeautifulSoup para analizar el HTML
    soup = BeautifulSoup(pagina_html, 'html.parser')

    print("_______________________________")

    try:
        nombre= driver.find_element(By.XPATH,'//*[@id="__next"]/main/div[3]/div/div/div/div[1]/div/div[1]/a/div')
        print(nombre.text)
        
        try:
            puntuacion= driver.find_element(By.XPATH,'//*[@id="__next"]/main/div[3]/div/div/div/div[1]/div/div[2]/div/span')
            print(puntuacion.text)

            print("_______________________________")

            # DEVOLVER TODOS LOS PARÁMETROS DE RENDIMIENTO DEL JUGADOR: encontrar todos los div con la clase "sc-fqkvVR sc-dcJsrY litZes eFJwJL"
            entradas = []
            goal_elements = soup.find_all('div', class_='sc-fqkvVR sc-dcJsrY litZes eFJwJL')
            for element in goal_elements:
                
                # Encuentra el div con la clase "sc-jEACwC hFGVAX" y el span con la clase "sc-jEACwC jnyhQn" dentro de este div
                div_goal = element.find('div', class_='sc-jEACwC hFGVAX')
                span_goal = element.find('span', class_='sc-jEACwC jnyhQn')
                if div_goal and span_goal:
                    print(div_goal.text)
                    print(span_goal.text)

                    estadisticas = {}
                    clave = div_goal.text
                    valor = span_goal.text
                    estadisticas[clave] = valor

                    entrada = {
                        clave: valor
                    }

                    # Agrega la entrada JSON a la lista
                    entradas.append(entrada)
                    
                nombreJson=nombre.text
                puntuaciónJson=puntuacion.text
        
                # Crear el diccionario para el jugador
                JsonJugador = {
                    nombreJson: {
                        "puntuacion": puntuaciónJson,
                        "estadisticas": entradas
                    }
                }
                
                performance_to_json(JsonJugador)

        except NoSuchElementException as e:
            print("Sin jugar")
            print("_______________________________")
            
            entradas = []
            estadisticas = {}
            clave = "Minutes played"
            valor = 0
            estadisticas[clave] = valor

            entrada = {
                clave: valor
            }

            # Agrega la entrada JSON a la lista
            entradas.append(entrada)
            
            nombreJson=nombre.text
            puntuaciónJson=None
        
            # Crear el diccionario para el jugador
            JsonJugador = {
                    nombreJson: {
                        "puntuacion": puntuaciónJson,
                        "estadisticas": entradas
                    }
            }
            
            performance_to_json(JsonJugador)
            
            return

    except NoSuchElementException as e:
        print("Entrenador")
    print("_______________________________")

##### - función para abrir un driver en Google Chrone, aceptar las cookies y saltar el mensaje de valorar el sitio web

In [5]:
def open_driver():
    # Crea una instancia del controlador del navegador
    driver = webdriver.Chrome()

    # Navega a la página web que deseas hacer scraping
    driver.get(partido)

    # Espera a que se cargue la página
    driver.implicitly_wait(45)

    # Encuentra el botón de "Consentir" 
    button = driver.find_element(By.XPATH, '//button[@aria-label="Consentir"]')
    # Haz clic en el botón de "Consentir" 
    button.click()
    try:
        # Encuentra el botón de "Ask me later" 
        button = driver.find_element(By.XPATH, '//*[@id="__next"]/div[3]/div[2]/button')
        # Haz clic en el botón de "Consentir" 
        button.click()
    except:
        pass


In [6]:
##############################################################################################################################################################
# FASE 1: Obtener la url de la pagina web de Sofaescore de todas los partidos de LaLiga                                                                      #
##############################################################################################################################################################

#######################################################################################################################################################
#  PARTE 1 : Mediante una request a la API de Sofaescore obtenemos el id de todos los equipos de LaLiga para posteriores cnsultas                     #
#######################################################################################################################################################

url = "https://sofascore.p.rapidapi.com/teams/search"

#equipos = ["Real Madrid", "Barcelona", "Atlético de Madrid", "Valencia", "Sevilla", "Villarreal", "Real Sociedad", "Real Betis", "Athletic Club", "Celta Vigo", "Almeria", "Getafe", "Mallorca", "Girona", "Granada", "Cadiz", "Alavés", "Rayo Vallecano", "Osasuna", "Las Palmas"]
equipos = ["Barcelona"]

ids_equipos = []

valor_round = None
slugJson = None

print("______________________________________________________________________________________________________________________________________________")  
print("Obteniendo id de todos los equipos de LaLiga... ")

for equipo in equipos:
    response = requests.get(url, headers=headers, params={"name": equipo})
    data = response.json()
    if data['teams']:
        equipo_id = data['teams'][0]['id']
        ids_equipos.append(equipo_id)
        print(f"GET ID: {equipo}: {equipo_id}")
        
print("______________________________________________________________________________________________________________________________________________")        

______________________________________________________________________________________________________________________________________________
Obteniendo id de todos los equipos de LaLiga... 
GET ID: Barcelona: 2817
______________________________________________________________________________________________________________________________________________


In [7]:
#######################################################################################################################################################
#  PARTE 2 : Mediante una request a la API de Sofaescore obtenemos informacion de todos los equipos de LaLiga                                         #
#######################################################################################################################################################

url = "https://sofascore.p.rapidapi.com/teams/get-last-matches"
ultimo_partido_equipos = []

print("______________________________________________________________________________________________________________________________________________")  
print("Obteniendo último partido de cada equipo...")

for team_id in ids_equipos:
    querystring = {"teamId": str(team_id), "pageIndex": "0"}

    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code == 200:

            data = response.json()
            
            num_partido = len(data['events'])-1
            
            nombre=data['events'][num_partido]['slug']
            print("GET:"+str(nombre))
            
            ulimo_partido=data['events'][num_partido]
            print(ulimo_partido)
            
            
            if ulimo_partido not in ultimo_partido_equipos:
                ultimo_partido_equipos.append(ulimo_partido)
                
            valor_round = data['events'][num_partido]['roundInfo']['round']
            #slugJson = data['events'][num_partido]['slug']
    else:
        print(f"Error al obtener datos para el equipo con ID {team_id}. Código de estado: {response.status_code}")
        
nombre_carpeta_jornada = "jornada_" + str(valor_round)
ruta_jornada = os.path.join("performance_jugadores_por_partidos", nombre_carpeta_jornada)

print("______________________________________________________________________________________________________________________________________________")   

______________________________________________________________________________________________________________________________________________
Obteniendo último partido de cada equipo...
GET:real-madrid-barcelona
{'tournament': {'name': 'LaLiga', 'slug': 'laliga', 'category': {'name': 'Spain', 'slug': 'spain', 'sport': {'name': 'Football', 'slug': 'football', 'id': 1}, 'id': 32, 'flag': 'spain', 'alpha2': 'ES'}, 'uniqueTournament': {'name': 'LaLiga', 'slug': 'laliga', 'primaryColorHex': '#2f4a89', 'secondaryColorHex': '#f4a32e', 'category': {'name': 'Spain', 'slug': 'spain', 'sport': {'name': 'Football', 'slug': 'football', 'id': 1}, 'id': 32, 'flag': 'spain', 'alpha2': 'ES'}, 'userCount': 873188, 'crowdsourcingEnabled': False, 'hasPerformanceGraphFeature': True, 'id': 8, 'country': {}, 'hasEventPlayerStatistics': True, 'displayInverseHomeAwayTeams': False}, 'priority': 588, 'id': 36}, 'roundInfo': {'round': 11}, 'customId': 'rgbsEgb', 'status': {'code': 100, 'description': 'Ended', 't

In [8]:
#######################################################################################################################################################       
#  PARTE 3 : Mediante los datos obtenidos de la API cosntruimos la url de cada patido de todos los equipos de LaLiga                                  #
#######################################################################################################################################################

ids_url = []
slugs_url= []
custom_ids_url= []

urls= []

for partido in ultimo_partido_equipos:
    
    id = partido.get('id')
    ids_url.append(id)
    
    slug = partido.get('slug')
    slugs_url.append(slug)
    
    customId = partido.get('customId')
    custom_ids_url.append(customId)

base_url = "https://www.sofascore.com/{}/{}#{}"

nume_urls = len(ids_url) 

print("______________________________________________________________________________________________________________________________________________")  
print("Generando urls del último partido de cada equipo...")

for i in range(nume_urls):
    url = base_url.format(slugs_url[i], custom_ids_url[i], ids_url[i])
    print(url)
    urls.append(url)

print("______________________________________________________________________________________________________________________________________________")   

______________________________________________________________________________________________________________________________________________
Generando urls del último partido de cada equipo...
https://www.sofascore.com/real-madrid-barcelona/rgbsEgb#11369369
______________________________________________________________________________________________________________________________________________


In [10]:
##############################################################################################################################################################
# FASE 2: "Acceder y hacer scraping de todos las urls de todos los partidos de LaLiga                                                                        #
##############################################################################################################################################################

#######################################################################################################################################################
#  PARTE 1 : obtener el performance de los 22 jugadores titulares del partido                                                                         # 
#    -abre la web                                                                                                                                     #
#    -acepta las cookies                                                                                                                              #
#    -hace click sobre de cada jugador para que emerja la tarjeta con los datos del performance del partido asociados a cada jugador y los extrae)    #
#######################################################################################################################################################
print("Starting scraper...")

for partido in urls:
    # Crea una instancia del controlador del navegador
    driver = webdriver.Chrome()
    
    # Maximizar la ventana del navegador
    driver.maximize_window()

    # Navega a la página web que deseas hacer scraping
    driver.get(partido)

    # Espera a que se cargue la página
    driver.implicitly_wait(45)

    # Encuentra el botón de "Consentir" 
    button = driver.find_element(By.XPATH, '//button[@aria-label="Consentir"]')
    # Haz clic en el botón de "Consentir" 
    button.click()
    try:
        # Encuentra el botón de "Ask me later" 
        button = driver.find_element(By.XPATH, '//*[@id="__next"]/div[3]/div[2]/button')
        # Haz clic en el botón de "Consentir" 
        button.click()
        
    except:
        pass
    time.sleep(45)
    # Encuentra el nombre del partido" 
    local = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/div[1]/div[1]/div[1]/div[1]/div/div[1]/div/a/div/div/bdi')
    visitante = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/div[1]/div[1]/div[1]/div[1]/div/div[3]/div/a/div/div/bdi')
    # Concatenar los nombres para formar slugJson
    slugJsonConcatenado = f"{local.text}_{visitante.text}"
    slugJson = slugJsonConcatenado.replace(" ", "_")
    
    driver.quit()
    
    print("Scraping", slugJson,"...")
   
    for i in range(22):

        # Crear carpeta para la jornada
        if not os.path.exists(ruta_jornada):
            os.makedirs(ruta_jornada)

        # Crea una instancia del controlador del navegador
        driver = webdriver.Chrome()
        
        # Maximizar la ventana del navegador
        driver.maximize_window()

        # Navega a la página web que deseas hacer scraping
        driver.get(partido)

        # Espera a que se cargue la página
        driver.implicitly_wait(45)

        # Encuentra el botón de "Consentir" 
        button = driver.find_element(By.XPATH, '//button[@aria-label="Consentir"]')
        # Haz clic en el botón de "Consentir" 
        button.click()

        try:
            # Encuentra el botón de "Ask me later" 
            button = driver.find_element(By.XPATH, '//*[@id="__next"]/div[3]/div[2]/button')
            # Haz clic en el botón de "Consentir" 
            button.click()
        except:
            pass
        
        # Encuentra todos los elementos <a> con la clase 'sc-3937c22d-0 jrbLdB'
        divJugadores = driver.find_elements(By.XPATH, '//a[@class="sc-3937c22d-0 jrbLdB"]')
        
        numTitulares=len(divJugadores)
        print(i+1,"/",numTitulares)
        divJugadores[i].click()
        time.sleep(45)
        obtener_informacion_jugador()

        driver.quit()

Starting scraper...
Scraping Barcelona_Real_Madrid ...
1 / 22
_______________________________
Marc-André ter Stegen
6.9
_______________________________
Minutes played
90'
Saves
2
Goals prevented
-0.68
Punches
0
Runs out (succ.)
0 (0)
High claims
1
Touches
40
Acc. passes
27/29 (93%)
Key passes
0
Crosses (acc.)
0 (0)
Long balls (acc.)
10 (8)
Clearances
0
Blocked shots
0
Interceptions
0
Tackles
0
Dribbled past
0
Ground duels (won)
0 (0)
Aerial duels (won)
0 (0)
Fouls
0
Was fouled
0
Shots on target
0
Shots off target
0
Shots blocked
0
Dribble attempts (succ.)
0 (0)
_______________________________
2 / 22
_______________________________
Ronald Araújo
6.8
_______________________________
Minutes played
90'
Goals
0
Expected Goals (xG)
0.05
Assists
0
Clearances
1
Blocked shots
1
Interceptions
1
Tackles
3
Dribbled past
0
Ground duels (won)
6 (3)
Aerial duels (won)
3 (1)
Possession lost
20
Fouls
2
Was fouled
0
Touches
71
Acc. passes
37/53 (70%)
Key passes
0
Crosses (acc.)
1 (0)
Long balls (acc.)
1

In [11]:
#######################################################################################################################################################
#  PARTE 2 :  obtener el performance de los jugadores que entraron de cambio al partido                                                               #
#######################################################################################################################################################
for partido in urls:   
    jugador=0

    while True:

        # Crea una instancia del controlador del navegador
        driver = webdriver.Chrome()
        
         # Maximizar la ventana del navegador
        driver.maximize_window()

        # Navega a la página web que deseas hacer scraping
        driver.get(partido)

        # Espera a que se cargue la página
        driver.implicitly_wait(45)

        #Encuentra el botón de "Consentir" 
        button = driver.find_element(By.XPATH, '//button[@aria-label="Consentir"]')
        # Haz clic en el botón de "Consentir" 
        button.click()
        try:
            # Encuentra el botón de "Ask me later" 
            button = driver.find_element(By.XPATH, '//*[@id="__next"]/div[3]/div[2]/button')
            # Haz clic en el botón de "Consentir" 
            button.click()
        except:
            pass

        # Encuentra todos los elementos 
        divSuplentes = driver.find_elements(By.XPATH, '//div[@display="flex" and contains(@class, "sc-fqkvVR sc-dcJsrY bASBgQ kHiXJe")]')

        tamaño_divSuplentes = len(divSuplentes)
        print(jugador+1,"/", tamaño_divSuplentes)

        divSuplentes[jugador].click()
        time.sleep(45)
        obtener_informacion_jugador()

        jugador += 1 
        
        if jugador >= len(divSuplentes):
            driver.quit()
            break

        driver.quit()
        
    #Obtener los jugadores del final de la lista que no se detectan en e while anterior
    jugador=0
    while True:

        # Crea una instancia del controlador del navegador
        driver = webdriver.Chrome()
        
         # Maximizar la ventana del navegador
        driver.maximize_window()

        # Navega a la página web que deseas hacer scraping
        driver.get(partido)

        # Espera a que se cargue la página
        driver.implicitly_wait(45)

        #Encuentra el botón de "Consentir" 
        button = driver.find_element(By.XPATH, '//button[@aria-label="Consentir"]')
        # Haz clic en el botón de "Consentir" 
        button.click()
        try:
            # Encuentra el botón de "Ask me later" 
            button = driver.find_element(By.XPATH, '//*[@id="__next"]/div[3]/div[2]/button')
            # Haz clic en el botón de "Consentir" 
            button.click()
        except:
            pass
        
        # Encuentra el elemento con la estructura dada
        divSuplentes = driver.find_elements(By.XPATH, '//div[@display="flex" and contains(@class, "sc-fqkvVR sc-dcJsrY bphgaj kHiXJe")]')

        

        tamaño_divSuplentes = len(divSuplentes)
        print(jugador+1,"/", tamaño_divSuplentes)

        divSuplentes[jugador].click()
        time.sleep(45)
        obtener_informacion_jugador()

        jugador += 1 
        
        if jugador >= len(divSuplentes):
            driver.quit()
            break

        driver.quit()

1 / 26
_______________________________
Entrenador
_______________________________
2 / 26
_______________________________
Robert Lewandowski
6.4
_______________________________
Minutes played
29'
Goals
0
Expected Goals (xG)
0.03
Assists
0
Expected Assists (xA)
0.01
Shots on target
0
Shots off target
1
Shots blocked
0
Dribble attempts (succ.)
0 (0)
Touches
21
Acc. passes
10/14 (71%)
Key passes
0
Crosses (acc.)
0 (0)
Long balls (acc.)
1 (0)
Ground duels (won)
2 (1)
Aerial duels (won)
1 (0)
Possession lost
8
Fouls
0
Was fouled
1
Clearances
0
Blocked shots
0
Interceptions
0
Tackles
0
Dribbled past
1
_______________________________
3 / 26
_______________________________
Oriol Romeu
6.7
_______________________________
Minutes played
18'
Goals
0
Expected Goals (xG)
0.03
Assists
0
Expected Assists (xA)
0.01
Touches
24
Acc. passes
14/17 (82%)
Key passes
1
Crosses (acc.)
0 (0)
Long balls (acc.)
0 (0)
Shots on target
0
Shots off target
0
Shots blocked
1
Dribble attempts (succ.)
1 (1)
Ground duels 